In [6]:
import os
import glob
import pandas as pd

def load_data(data_dir):
    # 결과를 저장할 리스트 초기화
    data = []

    # 상위 폴더 순회
    for speaker_dir in os.listdir(data_dir):
        speaker_path = os.path.join(data_dir, speaker_dir)
        if os.path.isdir(speaker_path):
            # 하위 폴더 내의 오디오 및 텍스트 파일 검색
            audio_files = glob.glob(os.path.join(speaker_path, '*.wav'))
            for audio_file in audio_files:
                # 오디오 파일에 대응하는 텍스트 파일 로드
                transcript_file = audio_file.replace('.wav', '.txt')
                if os.path.exists(transcript_file):
                    with open(transcript_file, 'r', encoding='utf-8') as f:
                        transcript = f.read().strip()

                    # 데이터셋 리스트에 저장
                    data.append({
                        'audio_path': audio_file,
                        'transcript': transcript
                    })

    # DataFrame으로 변환
    df = pd.DataFrame(data)
    return df

# 데이터 폴더 경로
data_dir = "/mnt/c/Users/tkd39/stt/1.Training/D03/J13/"
dataset = load_data(data_dir)

# 데이터 확인
print(dataset.head())


                                          audio_path  \
0  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
1  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
2  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
3  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
4  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   

                                          transcript  
0         o/ n/ 네 감사합니다. (NCS)/(엔씨에스) 교육과정 문의 체험입니다.  
1                                      o/ n/ 네 여보세요.  
2  o/ n/ 아 네 저기 그 (NCS)/(엔씨에스) 인사담당자 기본 심화과정 신청하고...  
3  o/ n/ 아 네 맞습니다. 홈페이지에 나와있는 어 네, 네. 기본과정이나 심화과정...  
4                                     o/ n/ 네 알겠습니다.  


In [8]:
dataset.to_csv('/mnt/datasets.csv')

In [10]:
dataset = pd.read_csv('/mnt/c/datasets.csv')

In [11]:
dataset = dataset.head(50000)

In [12]:
# PyTorch 관련
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence

# Transformers 관련 (Hugging Face 라이브러리)
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
    DataCollatorForSeq2Seq
)

# Datasets 및 평가 관련
from datasets import load_dataset, load_from_disk
import evaluate

# 시각화 및 Jupyter Notebook 관련
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 기타 유틸리티
from dataclasses import dataclass
from typing import Any, Dict, List, Union


In [13]:
import pandas as pd
import re
# 1. 텍스트 전처리 함수 정의
def clean_transcript(text):
    # 불필요한 접두사 제거
    text = re.sub(r'o/ n/ ', '', text)  # "o/ n/" 제거
    text = re.sub(r'\([^)]*\)', '', text)  # 괄호 안 텍스트 제거
    text = text.strip()  # 앞뒤 공백 제거
    return text

# 텍스트 전처리 적용
dataset['transcript'] = dataset['transcript'].apply(clean_transcript)

# 2. 데이터셋 확인
print(dataset.head())

   Unnamed: 0                                         audio_path  \
0           0  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
1           1  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
2           2  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
3           3  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   
4           4  /mnt/c/Users/tkd39/stt/1.Training/D03/J13/S000...   

                                          transcript  
0                          네 감사합니다. / 교육과정 문의 체험입니다.  
1                                            네 여보세요.  
2  아 네 저기 그 / 인사담당자 기본 심화과정 신청하고 싶은데 교육 시간이 어떻게 되...  
3  아 네 맞습니다. 홈페이지에 나와있는 어 네, 네. 기본과정이나 심화과정 / 다 어...  
4                                           네 알겠습니다.  


In [14]:
# 열 이름 변경 (Whisper가 요구하는 형식에 맞추기 위해)
dataset = dataset.rename(columns={'audio_path': 'audio', 'transcript': 'transcription'})

# 최종 데이터셋 저장
dataset.to_csv('/mnt/c/whisper_korean_cleaned_dataset.csv', index=False, encoding='utf-8')

In [16]:
# 데이터셋 로드
dataset = load_dataset('csv', data_files='/mnt/c/whisper_korean_cleaned_dataset.csv')


Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
# 오디오 열을 16kHz로 리샘플링
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))


In [18]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델과 프로세서 로드
model_name = "openai/whisper-tiny"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# 모델의 태스크와 언어 설정 (한국어: 'ko')
processor.feature_extractor.language = "ko"
processor.feature_extractor.task = "transcribe"  # 또는 "translate"

In [19]:
def prepare_dataset(batch):
    # 오디오 데이터 처리
    audio = batch["audio"]
    batch["input_features"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    
    # 텍스트에 언어 코드를 추가해 토크나이징
    batch["labels"] = processor.tokenizer(f"[KO] {batch['transcription']}").input_ids
    return batch

# 데이터셋 전처리 적용
processed_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"])


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [20]:
# 전처리된 데이터셋 저장
processed_dataset.save_to_disk("/mnt/dwhisper_ko_processed_dataset2")

Saving the dataset (0/97 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [2]:
from datasets import load_from_disk

# 저장된 데이터셋 로드
processed_dataset = load_from_disk("/mnt/dwhisper_ko_processed_dataset2")

Loading dataset from disk:   0%|          | 0/97 [00:00<?, ?it/s]

In [21]:
# 훈련 및 테스트 데이터셋 분할 (80% 훈련, 20% 테스트)
processed_dataset = processed_dataset["train"].train_test_split(test_size=0.1)

# 분할된 데이터셋 확인
print(processed_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5000
    })
})


In [22]:
from dataclasses import dataclass
import torch
from typing import Any, Dict, List, Union
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from torch.nn.utils.rnn import pad_sequence

@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: PreTrainedTokenizerBase
    padding_value: int = -100  # -100은 ignore_index로 자주 사용됨

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Union[torch.Tensor, Any]]:
        # input_features와 labels를 분리
        input_features = [torch.tensor(f["input_features"], dtype=torch.float) for f in features]
        labels = [torch.tensor(f["labels"], dtype=torch.long) for f in features]

        # input_features를 텐서로 변환
        batch = {
            "input_features": torch.stack(input_features),
        }

        # labels를 패딩 처리하여 텐서로 변환
        batch["labels"] = pad_sequence(labels, batch_first=True, padding_value=self.padding_value)

        return batch

# 데이터 콜레이터 생성
data_collator = DataCollatorSpeechSeq2Seq(processor=processor)



In [23]:
import evaluate

# 평가 메트릭 로드
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# 평가 함수 정의
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # id를 텍스트로 변환
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id  # ignore_index 처리
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    
    # 메트릭 계산
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer, "cer": cer}

In [24]:
from transformers import TrainerCallback
import matplotlib.pyplot as plt
from IPython.display import clear_output

# WER, CER를 저장할 리스트 초기화
train_losses, val_losses, wers, cers = [], [], [], []

class MetricsPlotCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        # 평가 단계에서 로스와 메트릭 업데이트
        logs = kwargs['metrics']
        train_losses.append(logs.get("loss", None))
        val_losses.append(logs.get("eval_loss", None))
        wers.append(logs.get("eval_wer", None))
        cers.append(logs.get("eval_cer", None))
        
        # 플롯 업데이트
        clear_output(wait=True)
        plt.figure(figsize=(10, 5))
        
        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label="Training Loss")
        plt.plot(val_losses, label="Validation Loss")
        plt.xlabel("Evaluation Step")
        plt.ylabel("Loss")
        plt.legend()
        plt.title("Training and Validation Loss")
        
        # WER and CER plot
        plt.subplot(1, 2, 2)
        plt.plot(wers, label="WER")
        plt.plot(cers, label="CER")
        plt.xlabel("Evaluation Step")
        plt.ylabel("Error Rate")
        plt.legend()
        plt.title("WER and CER")
        
        plt.show()

In [25]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# 학습 파라미터 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="/mnt/d/whisper_korean",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=3,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir='./logs',
)

# Trainer에 콜백 추가
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[MetricsPlotCallback()]  # 커스텀 콜백 추가
)



/home/shin/anaconda3/envs/tensor/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_17831/322104268.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/home/shin/anaconda3/envs/tensor/lib/python3.10/site-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

In [27]:
# 모델 저장
trainer.save_model("/mnt/whisper_korean_finetuned")

# 프로세서 저장 (토크나이저와 feature_extractor 포함)
processor.save_pretrained("/mnt/whisper_korean_finetuned")


[]

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 저장된 모델과 프로세서 로드
processor = WhisperProcessor.from_pretrained("/mnt/whisper_korean_finetuned")
model = WhisperForConditionalGeneration.from_pretrained("/mnt/whisper_korean_finetuned")

In [31]:
def evaluate_model(model, processor, dataset):
    references = []  # 실제 텍스트 (ground truth)
    predictions = [] # 모델 예측 텍스트

    # 평가 모드로 전환
    model.eval()
    model.to("cuda")  # 모델을 GPU로 이동

    for example in dataset:
        # 이미 전처리된 input_features 사용
        input_features = torch.tensor(example["input_features"]).unsqueeze(0).to("cuda")  # 차원 확장 및 GPU로 전송

        # 모델 예측 생성
        with torch.no_grad():
            predicted_ids = model.generate(input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # labels를 텍스트로 변환
        label_ids = example["labels"]
        label_ids = [id for id in label_ids if id != -100]  # ignore_index 값 제거
        reference = processor.decode(label_ids, skip_special_tokens=True)

        # 예측과 실제값 저장
        predictions.append(transcription)
        references.append(reference)

    # WER, CER 계산
    wer = wer_metric.compute(predictions=predictions, references=references)
    cer = cer_metric.compute(predictions=predictions, references=references)

    return {"WER": wer, "CER": cer}

In [33]:
# 현재 저장된 리스트 값들 출력
print("Training Losses:", train_losses)
print("Validation Losses:", val_losses)
print("WERs:", wers)
print("CERs:", cers)

Training Losses: [None, None, None]
Validation Losses: [0.4748903214931488, 0.39683955907821655, 0.38040995597839355]
WERs: [0.39762552846470145, 0.4058493079284184, 0.40411188973185846]
CERs: [0.26097289548635344, 0.2713437828118025, 0.2584957026362613]


In [34]:
results = evaluate_model(model, processor, test_dataset)
print("Evaluation Results:", results)

Evaluation Results: {'WER': 0.404030810216019, 'CER': 0.25844540430427976}
